http://localhost:8070

In [11]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os
#https://repo1.maven.org/maven2/com/github/housepower/clickhouse-native-jdbc/2.7.1/clickhouse-native-jdbc-2.7.1.jar
packages = [
    "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3",
    "com.clickhouse:clickhouse-jdbc:0.6.0",
    "com.clickhouse:clickhouse-http-client:0.6.0",
    "org.apache.httpcomponents.client5:httpclient5:5.3.1",
    "com.github.housepower:clickhouse-native-jdbc:2.7.1"
]
ram = 12
cpu = 12
# Define the application name and setup session
appName = "Connect To ClickHouse via PySpark"
spark = (SparkSession.builder
         .appName(appName)
         .config("spark.jars.packages", "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3,com.clickhouse:clickhouse-jdbc:0.6.0,com.clickhouse:clickhouse-http-client:0.6.0,org.apache.httpcomponents.client5:httpclient5:5.3.1,com.github.housepower:clickhouse-native-jdbc:2.7.1")
         .config("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
         .config("spark.sql.catalog.clickhouse.host", "34.118.0.142") # from GCP VM instances or other server ip
         .config("spark.sql.catalog.clickhouse.protocol", "http")
         .config("spark.sql.catalog.clickhouse.http_port", "8123")
         .config("spark.sql.catalog.clickhouse.user", "default")
         .config("spark.sql.catalog.clickhouse.password", "1234")
         .config("spark.sql.catalog.clickhouse.database", "default")
         .config("spark.spark.clickhouse.write.compression.codec", "lz4")
         .config("spark.clickhouse.read.compression.codec", "lz4")
         .config("spark.clickhouse.write.format", "arrow")
         #    .config("spark.clickhouse.write.distributed.convertLocal", "true") 
         #    .config("spark.clickhouse.write.repartitionNum", "1") 
         #.config("spark.clickhouse.write.maxRetry", "1000")
         #    .config("spark.clickhouse.write.repartitionStrictly", "true") 
         #    .config("spark.clickhouse.write.distributed.useClusterNodes", "false") 
         #.config("spark.clickhouse.write.batchSize", "650")
         .config("spark.sql.catalog.clickhouse.socket_timeout", "600000000")
         .config("spark.sql.catalog.clickhouse.connection_timeout", "600000000")
         .config("spark.sql.catalog.clickhouse.query_timeout", "600000000")
         .config("spark.clickhouse.options.socket_timeout", "600000000")
         .config("spark.clickhouse.options.connection_timeout", "600000000")
         .config("spark.clickhouse.options.query_timeout", "600000000")
         .config("spark.executor.memory", f"{ram}g")
         .config("spark.executor.cores", "5")
         .config("spark.driver.maxResultSize", f"{ram}g")
         .config("spark.driver.memory", f"{ram}g")
         .config("spark.executor.memoryOverhead", f"{ram}g")
         .config("spark.sql.debug.maxToStringFields", "100000")
         .getOrCreate()
         )
# spark.conf.set("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
# spark.conf.set("spark.sql.catalog.clickhouse.host", "127.0.0.1")
# spark.conf.set("spark.sql.catalog.clickhouse.protocol", "http")
# spark.conf.set("spark.sql.catalog.clickhouse.http_port", "8123")
# spark.conf.set("spark.sql.catalog.clickhouse.user", "default")
# spark.conf.set("spark.sql.catalog.clickhouse.password", "")
# spark.conf.set("spark.sql.catalog.clickhouse.database", "default")
spark.sql("use clickhouse")

DataFrame[]

In [14]:
spark.sql('show databases').show()

+------------------+
|         namespace|
+------------------+
|INFORMATION_SCHEMA|
|           default|
|    first_database|
|information_schema|
|            system|
+------------------+



In [15]:
spark.sql('CREATE DATABASE first_database').show()

AnalysisException: [SCHEMA_ALREADY_EXISTS] Cannot create schema `first_database` because it already exists.
Choose a different name, drop the existing schema, or add the IF NOT EXISTS clause to tolerate pre-existing schema.

In [16]:
df = spark.read.csv('/projects/app/stations.csv',header=True,inferSchema=True)

In [17]:
df.createOrReplaceTempView('df_sql')
spark.sql("""
INSERT INTO first_database.имятаблицы_котороесоздам  
SELECT * FROM df_sql
""")

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'и'.(line 2, pos 27)

== SQL ==

INSERT INTO first_database.имятаблицы_котороесоздам  
---------------------------^^^
SELECT * FROM df_sql


In [18]:
spark.sql("""
SELECT * FROM default.input_test_table
""").show()

+---+---+----------+----------+---------+--------------+---------------+-------------+--------------+--------+----------+-------------------+-----------+------------+---+---+----------+-------------+----------------+---+---+---+-------------+-------+
| id|age|height(cm)|weight(kg)|waist(cm)|eyesight(left)|eyesight(right)|hearing(left)|hearing(right)|systolic|relaxation|fasting blood sugar|Cholesterol|triglyceride|HDL|LDL|hemoglobin|Urine protein|serum creatinine|AST|ALT|Gtp|dental caries|smoking|
+---+---+----------+----------+---------+--------------+---------------+-------------+--------------+--------+----------+-------------------+-----------+------------+---+---+----------+-------------+----------------+---+---+---+-------------+-------+
|  0| 55|       165|        60|     81.0|           0.5|            0.6|          1.0|           1.0|     135|        87|                 94|        172|         300| 40| 75|      16.5|            1|             1.0| 22| 25| 27|            0|     

In [ ]:
q